# Generative AI Application Development

## Overview

- Start Serverless Warehouse

- Run initial setups

- Explore some Langchain tools/wrappers: PythonREPL, DuckDuckGo, Google Serper and ArXiv.

- Create a agent that decide what tool it should to use to answer the questions (as part of the output they generate a Python code that implement some tasks)

- Construct a second agent that answer questions about a Panda DataFrame of a dataset of paper reviews.

## Setup

In [0]:
# %run ./setup_env/agent_design

In [0]:
import pprint

## Checking tools

- https://docs.langchain.com/oss/python/integrations/tools

#### PythonREPL tool

In [0]:
from langchain_experimental.tools import PythonREPLTool

python_repl = PythonREPLTool()
python_repl.run('print("Hello!",  end="")')

Python REPL can execute arbitrary code. Use with caution.


'Hello!'


### DuckDuckGo search tool

In [0]:
from langchain_community.tools import DuckDuckGoSearchRun

search_ddg = DuckDuckGoSearchRun()
print(search_ddg.invoke('Top movies of Grammy Awards 2025'))

The 2025 Grammy Awards are here. Here's everything you need to know about the ceremony including the nominees, 
date, venue and how to watch. Grammy Awards 2025 : See the complete list of winners Beyoncé (finally!) wins Album 
of the Year, Kendrick Lamar snags 5 awards including Record and Song of the Year. TAGGED AS: Academy Awards , 
Awards , Awards Tour, Best Movies 2024, golden globes, Oscars, Top Movies Fresh off the Golden Globe awards , and 
Ryan Coogler's Sinners atop our leaderboard with 254 wins followed closely by One Battle After Another with 216 
wins. The 68th Grammy Awards took place in Los Angeles on Sunday night, with the biggest stars in the music 
industry gathering to celebrate the best songs, albums and artists of the year The "DJ Play A ... Discover the 
complete list of winners and nominees for the 2025 Grammy Awards . Explore every category, including Album of the 
Year and Best New Artis

In [0]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

search_ddg_wrapper = DuckDuckGoSearchAPIWrapper()
print(search_ddg_wrapper.run('Top movies of Grammy Awards 2025'))

Get full coverage of the 68th Annual GRAMMY Awards , including award categories, nominees, winners, and show 
highlights at GRAMMY .com Jan 27, 2026 · Fresh off the Golden Globe awards , and Ryan Coogler’s Sinners atop our 
leaderboard with 254 wins followed closely by One Battle After Another with 216 wins. As the two films have traded 
Best ... Feb 3, 2025 · The 2025 Grammy Awards are here. Here's everything you need to know about the ceremony 
including the nominees, date, venue and how to watch. The 2025 GRAMMYs marked a historic year for Music's Biggest 
Night, filled with monumental moments. Check out the complete list of winners and nominees from the 2025 GRAMMYs . 
1 day ago · The complete list of nominees and winners in the 95 categories presented at the 68th Grammy Awards on 
Sunday, Feb. 1, 2026 is below. Winners are noted in bold.

### Google Serper tool

- Need to create and configure SERPER_API_KEY

In [0]:
import os
os.environ['SERPER_API_KEY'] = dbutils.secrets.get(
    scope='serper',
    key='SERPER_API_KEY'
)

In [0]:
%sh
echo $SERPER_API_KEY

[REDACTED]


In [0]:
from langchain_community.utilities import GoogleSerperAPIWrapper

search_serper = GoogleSerperAPIWrapper()
print(search_serper.run('Top movies of Grammy Awards 2025'))

The 2025 GRAMMYs marked a historic year for Music's Biggest Night, filled with monumental moments. Check out the 
complete list of winners and nominees from the ... . Winner. Maestro. 6.5 (69K) Rate · . The Color Purple. 6.8 
(20K) Rate. (Various Artists) · . Deadpool & Wolverine. The Color Purple · 6.8 (17k). # 1 ; Deadpool & Wolverine · 
7.5 (552k). # 2 ; Twisters · 6.5 (190k). # 4 ; Saltburn · 7.0 (258k). # 5 ; American Fiction. 'American Symphony' 
Wins Best Music Film at 2025 GRAMMY Awards - Mercury Studios. Missing: Top | Show results with:Top. Dune 2 and 
Saturday Night were among my fav scores of the year. Batiste and Zimmer are well deserving winners. Record of the 
Year · "TEXAS HOLD 'EM" by Beyoncé · "Espresso" by Sabrina Carpenter · "360" by Charli xcx · "BIRDS OF A FEATHER" 
by Billie Eilish · "Not Like Us" by ... A look at the Best Music Film category at 2025 Grammys: films by or about 
Milli Vanilli, The Beatles, Taylor Swift and more are on the entry ... The complete list of nominees and winners in
the 95 categories presented at the 68th Grammy Awards on Sunday, Feb. 1, 2026 is below. General Field · "Not Like 
Us" – Kendrick Lamar · "Now and Then" – The Beatles · "Texas Hold 'Em" – Beyoncé · "Espresso" – Sabrina Carpenter ·
"360" – Charli XCX · " ... Missing: Top | Show results with:Top. “American Symphony” — Jon Batiste Matthew 
Heineman, video director; Lauren Domino, Matthew Heineman & Joedan Okun, video producers · “June” — ( ...

### ArXiv tool

In [0]:
from langchain_community.utilities import ArxivAPIWrapper

In [0]:
arxiv_tool = ArxivAPIWrapper()
docs = arxiv_tool.run('1706.03762')
print(docs)

Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, 
Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or convolutional neural networks 
in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an 
attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention 
mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show
these models to be superior in quality while being more parallelizable and requiring significantly less time to 
train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing
best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model 
establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a 
small fraction of the training costs of the best models from the literature. We show that the Transformer 
generalizes well to other tasks by applying it successfully to English constituency parsing both with large and 
limited training data.

## Create the 1st agent

### Define the brain of the agent

In [0]:
from langchain_community.chat_models import ChatDatabricks
# define a langchain chat model
endpoint_llm_1 = 'databricks-qwen3-next-80b-a3b-instruct'
llm_model_1 = ChatDatabricks(
	endpoint=endpoint_llm_1,
	max_tokens='400',
 	verbose=True
)


/home/spark-807bc36d-88b8-45a4-84eb-8e/.ipykernel/3333/command-4841382598003463-2071245504:4: LangChainDeprecationWarning: The class `ChatDatabricks` was deprecated in LangChain 0.3.3 and will be removed in 1.0. An updated version of the class exists in the `databricks-langchain package and should be used instead. To use it run `pip install -U `databricks-langchain` and import as `from `databricks_langchain import ChatDatabricks``.
  llm_model_1 = ChatDatabricks(


### Define the tools available to the agent

In [0]:
# ?PythonREPLTool

In [0]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_experimental.tools import PythonREPLTool

from langchain.tools import tool
from langchain.agents import create_agent

arxiv_wrapper = ArxivAPIWrapper()
search_ddg_wrapper = DuckDuckGoSearchAPIWrapper()
python_repl = PythonREPLTool()

@tool
def arxiv_tool(paper_desc: str) -> str:
    '''Search papers in the Arxiv.'''
    return arxiv_wrapper.run(paper_desc)

@tool
def search_tool(terms: str) -> str:
    '''Search terms in the Web.'''
    return search_ddg_wrapper.run(terms)

@tool
def python_tool(terms: str) -> str:
    '''Execute arbitrary Python codes.'''
    name='python_repl',
    description='A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.',
    func=python_repl.run,
    return python_repl.run(terms)


# set of tools
tools = [python_tool, arxiv_tool, search_tool]

### Define logic of the plan (ReAct)

In [0]:
from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can.You have acces to the following tools: [{tools}].

Use the following format:

Question: Is the input question that you must answer.
Though: You should always think about what to do.
Action: The action to take should be one of [{tools}].
Action_Input: The input to the action.
Observation: The result of the actions.
... (this Though/Action/Action_Input/Observation can repeat N times)
Though: I now know the final answer.
Final Answer: The final answer to the origial input question.

Question: {input}
Though: {agent_scratchpad}
'''
prompt = PromptTemplate.from_template(template)

In [0]:
from langgraph.prebuilt import create_react_agent

agent1 = create_react_agent(
    model=llm_model_1,
    tools=tools,
    # checkpointer=memory,
)

/home/spark-807bc36d-88b8-45a4-84eb-8e/.ipykernel/3333/command-5906691721714227-2037726547:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent1 = create_react_agent(


In [0]:
config = {
    'configurable': {'thread_id': '2026'}
}

human_input = '''What could be a nice paper to read about machine learning? Answer this question following the steps below.
Step 1: Decide which paper you would recommend.
Step 2: Show me the abstract of this paper.
Step 3: Collect data about the suggested paper using search tool.
Step 4: Generate the Python code to make a *word cloud* graph, using *wordcloud* library, with the words of the paper abstractof Step 3. If you do not encouter data at Step 3, use some dummy data to make the Python word cloud and then include a graph caption with the label "Using dummy data". 
... (Avoid use ``` for python code. Input should be sanitized by a convenient preprocessing that you suggest. If the input starts with "python", remove this word. The output must be a Python code.)
Final step: Tell me a curiosity about the paper that have you chosen.
...(Don't forget to generate the Python code for the word cloud.)
'''
input = {
    'messages': [('human', human_input)]
}

res1 = agent1.invoke(input=input, config=config, verbose=True, handle_parsing_error=True)

In [0]:
print(res1)

{
    'messages': [
        HumanMessage(
            content='What could be a nice paper to read about machine learning? Answer this question following the 
steps below.\nStep 1: Decide which paper you would recommend.\nStep 2: Show me the abstract of this paper.\nStep 3:
Collect data about the suggested paper using search tool.\nStep 4: Generate the Python code to make a *word cloud* 
graph, using *wordcloud* library, with the words of the paper abstractof Step 3. If you do not encouter data at 
Step 3, use some dummy data to make the Python word cloud and then include a graph caption with the label "Using 
dummy data". \n... (Avoid use ``` for python code. Input should be sanitized by a convenient preprocessing that you
suggest. If the input starts with "python", remove this word. The output must be a Python code.)\nFinal step: Tell 
me a curiosity about the paper that have you chosen.\n...(Don\'t forget to generate the Python code for the word 
cloud.)\n',
            additional_kwargs={},
            response_metadata={},
            id='fb370895-5f4d-478e-88b8-8a80047f5773'
        ),
        AIMessage(
            content='',
            additional_kwargs={
                'tool_calls': [
                    {
                        'id': 'call_f2cf4631-fc3d-4fe1-ab8d-242c3c940b69',
                        'type': 'function',
                        'function': {'name': 'arxiv_tool', 'arguments': '{"paper_desc": "machine learning"}'}
                    }
                ]
            },
            response_metadata={'prompt_tokens': 476, 'completion_tokens': 31, 'total_tokens': 507},
            id='lc_run--019c2502-059d-7562-a333-203d2f367d3f-0',
            tool_calls=[
                {
                    'name': 'arxiv_tool',
                    'args': {'paper_desc': 'machine learning'},
                    'id': 'call_f2cf4631-fc3d-4fe1-ab8d-242c3c940b69',
                    'type': 'tool_call'
                }
            ],
            invalid_tool_calls=[]
        ),
        ToolMessage(
            content='Published: 2023-06-07\nTitle: Changing Data Sources in the Age of Machine Learning for 
Official Statistics\nAuthors: Cedric De Boom, Michael Reusens\nSummary: Data science has become increasingly 
essential for the production of official statistics, as it enables the automated collection, processing, and 
analysis of large amounts of data. With such data science practices in place, it enables more timely, more 
insightful and more flexible reporting. However, the quality and integrity of data-science-driven statistics rely 
on the accuracy and reliability of the data sources and the machine learning techniques that support them. In 
particular, changes in data sources are inevitable to occur and pose significant risks that are crucial to address 
in the context of machine learning for official statistics.\n  This paper gives an overview of the main risks, 
liabilities, and uncertainties associated with changing data sources in the context of machine learning for 
official statistics. We provide a checklist of the most prevalent origins and causes of changing data sources; not 
only on a technical level but also regarding ownership, ethics, regulation, and public perception. Next, we 
highlight the repercussions of changing data sources on statistical reporting. These include technical effects such
as concept drift, bias, availability, validity, accuracy and completeness, but also the neutrality and potential 
discontinuation of the statistical offering. We offer a few important precautionary measures, such as enhancing 
robustness in both data sourcing and statistical techniques, and thorough monitoring. In doing so, machine 
learning-based official statistics can maintain integrity, reliability, consistency, and relevance in 
policy-making, decision-making, and public discourse.\n\nPublished: 2021-01-07\nTitle: DOME: Recommendations for 
supervised machine learning validation in biology\nAuthors: Ia

<img src="./imgs/code_output.png">

## Construct another agent

### Load data

In [0]:
import json
import pandas as pd

# define constants
DATASET_PATH = '/Volumes/studies/databricks-dev/data_json/reviews.json'
EXPORT_TABLES_TO_DATABRICKS = False


# load json dataset
with open(DATASET_PATH, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# make some adjusts to the dataset
df = pd.json_normalize(dataset['paper'])

df = pd.json_normalize(
    dataset,
    record_path=['paper', 'review'],  # take columns paper and review; each item in review is mapped to a row.
    meta=[['paper', 'id']],  # add paper.id as column
)

df.rename(columns={'paper.id': 'paper_id'}, inplace=True)

print(df.shape)
df.head()

(405, 9)

,confidence,evaluation,id,lan,orientation,remarks,text,timespan,paper_id
0,4,1,1,es,0,,- El artículo aborda un problema contingente y...,2010-07-05,1
1,4,1,2,es,1,,El artículo presenta recomendaciones prácticas...,2010-07-05,1
2,5,1,3,es,1,,- El tema es muy interesante y puede ser de mu...,2010-07-05,1
3,4,2,1,es,1,,Se explica en forma ordenada y didáctica una e...,2010-07-05,2
4,4,2,2,es,0,,,2010-07-05,2


### Define the agent

In [0]:
from langchain_community.chat_models import ChatDatabricks
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

prefix = '''Input should be sanitized by a convenient preprocessing that you suggest. If the input starts with "python", remove this word. You must to use the Pandas DataFrame dataset available to answer the input questions.
'''

# define a langchain chat model
endpoint_llm_2 = 'databricks-llama-4-maverick'
llm_model_2 = ChatDatabricks(
	endpoint=endpoint_llm_2,
	max_tokens='400',
	allow_dangerous_code=True,
    verbose=True
)


In [0]:
agent2 = create_pandas_dataframe_agent(
    llm=llm_model_2,
    df=df,
    prefix=prefix,
    allow_dangerous_code=True,
    verbose=True,
)

## Q/A session with the 2nd agent

In [0]:
df.head()

,confidence,evaluation,id,lan,orientation,remarks,text,timespan,paper_id
0,4,1,1,es,0,,- El artículo aborda un problema contingente y...,2010-07-05,1
1,4,1,2,es,1,,El artículo presenta recomendaciones prácticas...,2010-07-05,1
2,5,1,3,es,1,,- El tema es muy interesante y puede ser de mu...,2010-07-05,1
3,4,2,1,es,1,,Se explica en forma ordenada y didáctica una e...,2010-07-05,2
4,4,2,2,es,0,,,2010-07-05,2


In [0]:
agent2.invoke('What was the most confidence paper reviews?')



> Entering new AgentExecutor chain...
Thought: To find the most confidence paper reviews, I need to look at the 'confidence' column in the DataFrame and determine the maximum value.

Action: python_repl_ast
Action Input: `df['confidence'].max()`TypeError: '>=' not supported between instances of 'str' and 'float'The error indicates that the 'confidence' column contains non-numeric data, which is preventing the max() function from working correctly. I need to check the data type of the 'confidence' column and clean it if necessary.

Action: python_repl_ast
Action Input: `df['confidence'].dtype`objectThe 'confidence' column is of object type, indicating it contains non-numeric data. I need to convert this column to a numeric type to perform the max() operation.

Action: python_repl_ast
Action Input: `df['confidence'] = pd.to_numeric(df['confidence'], errors='coerce')`NameError: name 'pd' is not definedTo fix the NameError, I need to import the pandas library and possibly reassign it to 

{'input': 'What was the most confidence paper reviews?',
 'output': 'The most confidence paper reviews had a confidence of 5.0, and there were 47 such reviews.'}

In [0]:
agent2.invoke('Which languages are there in the paper reviews dataset?')



> Entering new AgentExecutor chain...
Thought: To determine the languages present in the paper reviews dataset, I need to examine the 'lan' column of the DataFrame. First, I should verify that the DataFrame is loaded and available for use.

Action: python_repl_ast
Action Input: `print(df.head())`   confidence evaluation  id lan orientation remarks  \
0         4.0          1   1  es           0           
1         4.0          1   2  es           1           
2         5.0          1   3  es           1           
3         4.0          2   1  es           1           
4         4.0          2   2  es           0           

                                                text    timespan paper_id  
0  - El artículo aborda un problema contingente y...  2010-07-05        1  
1  El artículo presenta recomendaciones prácticas...  2010-07-05        1  
2  - El tema es muy interesante y puede ser de mu...  2010-07-05        1  
3  Se explica en forma ordenada y didáctica una e...  2010-0

{'input': 'Which languages are there in the paper reviews dataset?',
 'output': "The languages present in the paper reviews dataset are Spanish ('es') and English ('en')."}

In [0]:
agent2.invoke('Which period had the worst results in paper evaluations?')



> Entering new AgentExecutor chain...
Thought: To determine the period with the worst results in paper evaluations, we first need to understand the structure of the given DataFrame and identify the relevant columns. The DataFrame contains information about paper evaluations, including the 'timespan' (which could be interpreted as the period or date of evaluation) and 'evaluation' (which is the evaluation score). We need to analyze the 'evaluation' scores in relation to the 'timespan'.

Action: python_repl_ast
Action Input: `df.groupby('timespan')['evaluation'].mean().idxmin()`TypeError: agg function failed [how->mean,dtype->object]The error indicates that the 'evaluation' column is of object type, not numeric, so we cannot directly calculate the mean. We need to convert it to a numeric type first. Let's check the unique values in the 'evaluation' column to understand its content.

Action: python_repl_ast
Action Input: `df['evaluation'].unique()`['1' '2' '0' '-2' '-1']The 'evaluation'

{'input': 'Which period had the worst results in paper evaluations?',
 'output': '2014-07-05'}

In [0]:
agent2.invoke('Tell me a curiosity about any paper review of this dataset.')



> Entering new AgentExecutor chain...
Thought: To find a curiosity about any paper review in the dataset, I should first examine the available columns and data in the DataFrame. The `print(df.head())` output provides a snapshot of the data, including columns like `text`, `lan`, `confidence`, and `paper_id`. A curiosity could be related to the distribution of languages or the average confidence level in the reviews.

Action: python_repl_ast
Action Input: `df.describe()`       confidence  evaluation          id
count  403.000000  405.000000  405.000000
mean     3.573201    0.182716    1.824691
std      0.844341    1.502868    0.821362
min      1.000000   -2.000000    1.000000
25%      3.000000   -1.000000    1.000000
50%      4.000000    0.000000    2.000000
75%      4.000000    2.000000    2.000000
max      5.000000    2.000000    4.000000The `df.describe()` output provides summary statistics for the numerical columns in the DataFrame. To find a curiosity about any paper review, I can

{'input': 'Tell me a curiosity about any paper review of this dataset.',
 'output': 'One curiosity is that some papers, like `paper_id` 54, have very high average confidence levels (5.0) in their reviews. The review for this paper comments on its good presentation but suggests that it is too long.'}

<img src="./imgs/agent_output.png">